In [1]:
import snowflake
from snowflake.snowpark import Session
# Snowpark for Python
from snowflake.snowpark.types import DoubleType
import snowflake.snowpark.functions as F


import os
from dotenv import load_dotenv

In [2]:
# Load environment variables from .env file
load_dotenv()
account = os.getenv('ACCOUNT')
password =os.getenv('PASSWORD')
user = os.getenv('USER')
warehouse = os.getenv('WAREHOUSE')
database = os.getenv('DATABASE')
schema = os.getenv('SCHEMA')

In [3]:
connections = snowflake.connector.connect(user = user,
                                  password = password,
                                  account = account,
                                  database = database,
                                  schema = schema)
cursor = connections.cursor()

In [4]:
CONNECTION_PARAMETERS = {
    "account": account,
    "user": user,
    "password":password,
    # "role": "test_role",
    "database": database,
    "warehouse": warehouse,
    "schema": schema
}

In [5]:
session = Session.builder.configs(CONNECTION_PARAMETERS).create()

### Use the Snowpark DataFrame Reader to read in data from the externally staged diamonds CSV file

In [6]:

# Create a Snowpark DataFrame that is configured to load data from the CSV file
# We can now infer schema from CSV files.
diamonds_df = session.read.options({"field_delimiter": ",",
                                    "field_optionally_enclosed_by": '"',
                                    "infer_schema": True,
                                    "parse_header": True}).csv("@DIAMONDS_ASSETS")

diamonds_df.show()

------------------------------------------------------------------------------------------------
|"carat"  |"cut"      |"color"  |"clarity"  |"depth"  |"table"  |"price"  |"x"   |"y"   |"z"   |
------------------------------------------------------------------------------------------------
|0.23     |Ideal      |E        |SI2        |61.5     |55.0     |326      |3.95  |3.98  |2.43  |
|0.21     |Premium    |E        |SI1        |59.8     |61.0     |326      |3.89  |3.84  |2.31  |
|0.23     |Good       |E        |VS1        |56.9     |65.0     |327      |4.05  |4.07  |2.31  |
|0.29     |Premium    |I        |VS2        |62.4     |58.0     |334      |4.20  |4.23  |2.63  |
|0.31     |Good       |J        |SI2        |63.3     |58.0     |335      |4.34  |4.35  |2.75  |
|0.24     |Very Good  |J        |VVS2       |62.8     |57.0     |336      |3.94  |3.96  |2.48  |
|0.24     |Very Good  |I        |VVS1       |62.3     |57.0     |336      |3.95  |3.98  |2.47  |
|0.26     |Very Good  |H      

In [7]:
# Look at descriptive stats on the DataFrame
diamonds_df.describe().show()

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"SUMMARY"  |"carat"             |"cut"      |"color"  |"clarity"  |"depth"             |"table"             |"price"             |"x"                 |"y"                |"z"                 |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|count      |53940.0             |53940      |53940    |53940      |53940.0             |53940.0             |53940.0             |53940.0             |53940.0            |53940.0             |
|mean       |0.79793975          |NULL       |NULL     |NULL       |61.7494049          |57.4571839          |3932.799722         |5.73115721          |5.73452595         |3.53873378          |
|stddev     |0.474011244381396

In [11]:
print(diamonds_df.schema)

StructType([StructField('"carat"', DecimalType(3, 2), nullable=True), StructField('"cut"', StringType(), nullable=True), StructField('"color"', StringType(), nullable=True), StructField('"clarity"', StringType(), nullable=True), StructField('"depth"', DecimalType(3, 1), nullable=True), StructField('"table"', DecimalType(3, 1), nullable=True), StructField('"price"', LongType(), nullable=True), StructField('"x"', DecimalType(4, 2), nullable=True), StructField('"y"', DecimalType(4, 2), nullable=True), StructField('"z"', DecimalType(4, 2), nullable=True)])


In [9]:
diamonds_df.columns

['"carat"',
 '"cut"',
 '"color"',
 '"clarity"',
 '"depth"',
 '"table"',
 '"price"',
 '"x"',
 '"y"',
 '"z"']

# Data cleaning
First, let's force headers to uppercase using Snowpark DataFrame operations for standardization when columns are later written to a Snowflake table.

In [8]:
# Force headers to uppercase
for colname in diamonds_df.columns:
    if colname == '"table"':
       new_colname = "TABLE_PCT"
    else:
        new_colname = str.upper(colname)
    diamonds_df = diamonds_df.with_column_renamed(colname, new_colname)

diamonds_df

In [9]:
def fix_values(columnn):
    return F.upper(F.regexp_replace(F.col(columnn), '[^a-zA-Z0-9]+', '_'))

for col in ["CUT"]:
    diamonds_df = diamonds_df.with_column(col, fix_values(col))

diamonds_df.show()

----------------------------------------------------------------------------------------------------
|"CARAT"  |"COLOR"  |"CLARITY"  |"DEPTH"  |"TABLE_PCT"  |"PRICE"  |"X"   |"Y"   |"Z"   |"CUT"      |
----------------------------------------------------------------------------------------------------
|0.23     |E        |SI2        |61.5     |55.0         |326      |3.95  |3.98  |2.43  |IDEAL      |
|0.21     |E        |SI1        |59.8     |61.0         |326      |3.89  |3.84  |2.31  |PREMIUM    |
|0.23     |E        |VS1        |56.9     |65.0         |327      |4.05  |4.07  |2.31  |GOOD       |
|0.29     |I        |VS2        |62.4     |58.0         |334      |4.20  |4.23  |2.63  |PREMIUM    |
|0.31     |J        |SI2        |63.3     |58.0         |335      |4.34  |4.35  |2.75  |GOOD       |
|0.24     |J        |VVS2       |62.8     |57.0         |336      |3.94  |3.96  |2.48  |VERY_GOOD  |
|0.24     |I        |VVS1       |62.3     |57.0         |336      |3.95  |3.98  |2.47  |VER

In [10]:
list(diamonds_df.schema)

[StructField('CARAT', DecimalType(3, 2), nullable=True),
 StructField('COLOR', StringType(), nullable=True),
 StructField('CLARITY', StringType(), nullable=True),
 StructField('DEPTH', DecimalType(3, 1), nullable=True),
 StructField('TABLE_PCT', DecimalType(3, 1), nullable=True),
 StructField('PRICE', LongType(), nullable=True),
 StructField('X', DecimalType(4, 2), nullable=True),
 StructField('Y', DecimalType(4, 2), nullable=True),
 StructField('Z', DecimalType(4, 2), nullable=True),
 StructField('CUT', StringType(), nullable=True)]

In [12]:
for colname in ["CARAT", "X", "Y", "Z", "DEPTH", "TABLE_PCT"]:
    diamonds_df = diamonds_df.with_column(colname, diamonds_df[colname].cast(DoubleType()))

diamonds_df.show()

----------------------------------------------------------------------------------------------------
|"COLOR"  |"CLARITY"  |"PRICE"  |"CUT"      |"CARAT"  |"X"   |"Y"   |"Z"   |"DEPTH"  |"TABLE_PCT"  |
----------------------------------------------------------------------------------------------------
|E        |SI2        |326      |IDEAL      |0.23     |3.95  |3.98  |2.43  |61.5     |55.0         |
|E        |SI1        |326      |PREMIUM    |0.21     |3.89  |3.84  |2.31  |59.8     |61.0         |
|E        |VS1        |327      |GOOD       |0.23     |4.05  |4.07  |2.31  |56.9     |65.0         |
|I        |VS2        |334      |PREMIUM    |0.29     |4.2   |4.23  |2.63  |62.4     |58.0         |
|J        |SI2        |335      |GOOD       |0.31     |4.34  |4.35  |2.75  |63.3     |58.0         |
|J        |VVS2       |336      |VERY_GOOD  |0.24     |3.94  |3.96  |2.48  |62.8     |57.0         |
|I        |VVS1       |336      |VERY_GOOD  |0.24     |3.95  |3.98  |2.47  |62.3     |57.0 

In [13]:

diamonds_df.write.mode('overwrite').save_as_table('diamonds')